# 1. Import Packages

In [ ]:
%pip install supabase

In [2]:
%pip install psycopg2
%pip install sqlalchemy

   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   --- ------------------------------------ 0.3/2.7 MB ? eta -:--:--
   --------------- ------------------------ 1.0/2.7 MB 2.5 MB/s eta 0:00:01
   ----------------------- ---------------- 1.6/2.7 MB 2.7 MB/s eta 0:00:01
   ---------------------------------------- 2.7/2.7 MB 3.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
import psycopg2

# 2. Connect to supabase (Database)

In [ ]:
# 1. Connection string to supabase
# settings -> database -> connection string -> URI
conn_string = "postgresql://postgres:@db.bxtneetzpubijmukvapq.supabase.co:5432/postgres"

In [4]:
# 2. Cretae engine
engine = create_engine(conn_string)

In [5]:
# Test connection
try:
    conn = engine.connect()
    print("✅ Connected to Supabase!")
    conn.close()
except Exception as e:
    print(f"❌ Connection failed: {e}")

✅ Connected to Supabase!


# 3. Load Data

## 3.1 Check Table on Database

In [8]:
query_check = """
SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'public'
ORDER BY table_name;
"""

tables = pd.read_sql(query_check, engine)   
print(tables)

                table_name
0      competencies_yearly
1                dim_areas
2            dim_companies
3   dim_competency_pillars
4          dim_departments
5         dim_directorates
6            dim_divisions
7            dim_education
8               dim_grades
9               dim_majors
10           dim_positions
11               employees
12             papi_scores
13      performance_yearly
14          profiles_psych
15               strengths


In [10]:
query = """
SELECT *
FROM employees
LIMIT 5;
""" 
print(pd.read_sql(query, engine))

  employee_id                  fullname     nip  company_id  area_id  \
0   EMP100000            Rendra Pratama  806137           1        4   
1   EMP100001            Wulan Setiawan  476388           4        4   
2   EMP100002  Julia Jatmiko Situmorang  941921           1        3   
3   EMP100003                 Oka Halim  751615           1        3   
4   EMP100004               Dwi Pratama  443809           3        4   

   position_id  department_id  division_id  directorate_id  grade_id  \
0            1              6            3               3         2   
1            4              5            5               3         1   
2            5              1            1               3         1   
3            4              2            1               1         2   
4            6              5            3               3         1   

   education_id  major_id  years_of_service_months  
0             3         1                       64  
1             1         2   

Alias Dictionary
e : employees
c : dim_companies
a : dim_area
p : dim_positions
d : dim_departement
div : dim_divisions
dir : dim_directorate
grd : dim_grades
edu : dim_educations
mjr : dim_majors
str : strengths
comp : competencies_yearly
papi : papi_scores

In [ ]:
data="""
SELECT
    -- Employee Data (from employees table)
    e.employee_id, -- Employee ID
    e.fullname, -- Employee name
    e.nip, -- Employee NIP
    e.years_of_service_months, -- Employee years of service in months

    -- Dimension Data (from dim_tables)
    c.name as company_name, -- company name
    a.name as area, -- area name
    p.name as position, -- employee position
    d.name as departement_name, -- departement name
    div.name as division, -- division name
    dir.name as directorate, -- employee directorate
    grd.name as grade,
    edu.name as education,
    mjr.name as major,

    -- Performance Data (from performance_yearly table)
    perf.year as performance_year,
    perf.rating as performance_rating,

    -- Psychological Profile Data (from profiles_psych table)
    prof.pauli,
    prof.faxtor,
    prof.disc,
    prof.disc_word,
    prof.mbti,
    prof.iq,
    prof.gtq,
    prof.tiki,

    -- Strength (Strengths of each employee)
    str.rank as strength_rank,
    str.theme as strength_theme,

    -- competency_yearly
    comp.score as competency_score,
    comp.pillar_code as competency_pillar_code,
    comp.year as competency_year,

    --PAPI Score
    papi.scale_code as papi_scale_code,
    papi.score as papi_score


FROM employees e
    LEFT JOIN performance_yearly perf ON e.employee_id = perf.employee_id
    LEFT JOIN dim_departments d ON e.department_id = d.department_id
    LEFT JOIN profiles_psych prof ON e.employee_id = prof.employee_id
    LEFT JOIN dim_companies c ON e.company_id = c.company_id
    LEFT JOIN dim_areas a ON e.area_id = a.area_id
    LEFT JOIN dim_positions p ON e.position_id = p.position_id
    LEFT JOIN dim_divisions div ON e.division_id = div.division_id
    LEFT JOIN dim_directorates dir ON e.directorate_id = dir.directorate_id
    LEFT JOIN dim_grades grd ON e.grade_id = grd.grade_id
    LEFT JOIN dim_education edu ON e.education_id = edu.education_id
    LEFT JOIN dim_majors mjr ON e.major_id = mjr.major_id
    LEFT JOIN strengths str ON e.employee_id = str.employee_id
    LEFT JOIN competencies_yearly comp ON e.employee_id = comp.employee_id
    LEFT JOIN papi_scores papi ON e.employee_id = papi.employee_id
    
    ORDER BY e.employee_id, perf.year DESC;

    """


## 3.2 Load Main Data

In [24]:
main_data="""
SELECT
    -- Employee Data (from employees table)
    e.employee_id, -- Employee ID
    e.fullname, -- Employee name
    e.nip, -- Employee NIP
    e.years_of_service_months, -- Employee years of service in months

    -- Dimension Data (from dim_tables)
    c.name as company_name, -- company name
    a.name as area, -- area name
    p.name as position, -- employee position
    d.name as departement_name, -- departement name
    div.name as division, -- division name
    dir.name as directorate, -- employee directorate
    grd.name as grade,
    edu.name as education,
    mjr.name as major,

    -- Psychological Profile Data (from profiles_psych table)
    prof.pauli,
    prof.faxtor,
    prof.disc,
    prof.disc_word,
    prof.mbti,
    prof.iq,
    prof.gtq,
    prof.tiki



FROM employees e

    LEFT JOIN dim_departments d ON e.department_id = d.department_id
    LEFT JOIN profiles_psych prof ON e.employee_id = prof.employee_id
    LEFT JOIN dim_companies c ON e.company_id = c.company_id
    LEFT JOIN dim_areas a ON e.area_id = a.area_id
    LEFT JOIN dim_positions p ON e.position_id = p.position_id
    LEFT JOIN dim_divisions div ON e.division_id = div.division_id
    LEFT JOIN dim_directorates dir ON e.directorate_id = dir.directorate_id
    LEFT JOIN dim_grades grd ON e.grade_id = grd.grade_id
    LEFT JOIN dim_education edu ON e.education_id = edu.education_id
    LEFT JOIN dim_majors mjr ON e.major_id = mjr.major_id
   
    ORDER BY e.employee_id DESC;

    """


In [25]:
data_main = pd.read_sql(main_data, engine)
data_main.head()

,employee_id,fullname,nip,years_of_service_months,company_name,area,position,departement_name,division,directorate,...,education,major,pauli,faxtor,disc,disc_word,mbti,iq,gtq,tiki
0,EMP101999,Adit Mahendra,432959,58,PT Aurora Beauty Indonesia,Store,Data Analyst,R&D,Talent Management,Technology,...,S1,Business,76,68,DC,Dominant-Conscientious,INFP,90.0,25.0,8
1,EMP101998,Kurnia Subekti,577048,64,PT Lumo Cosmetics Asia,Plant,Sales Supervisor,Operations,Talent Management,Technology,...,D3,Business,42,21,CS,Conscientious-Steadiness,None,NaN,19.0,2
2,EMP101997,Satya Halim Putra,465240,16,PT Aurora Beauty Indonesia,Plant,Brand Executive,Marketing,Product Dev,Technology,...,SMA,Engineering,23,44,DC,Dominant-Conscientious,ISTP,NaN,NaN,1
3,EMP101996,Adit Maulana,361572,53,PT Mandala Distribution Center,HQ,Data Analyst,Operations,Digital Marketing,HR & Corp Affairs,...,S1,Business,52,78,DS,Dominant-Steadiness,INTP,NaN,24.0,6
4,EMP101995,Utami Kusuma Hidayat,791731,50,PT Mandala Distribution Center,Store,Sales Supervisor,HR,Operations,Commercial,...,S2,Economics,26,42,SC,Steadiness-Conscientious,ENTP,134.0,41.0,8


In [27]:
data_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2010 entries, 0 to 2009
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   employee_id              2010 non-null   object 
 1   fullname                 2010 non-null   object 
 2   nip                      2010 non-null   int64  
 3   years_of_service_months  2010 non-null   int64  
 4   company_name             2010 non-null   object 
 5   area                     2010 non-null   object 
 6   position                 2010 non-null   object 
 7   departement_name         2010 non-null   object 
 8   division                 2010 non-null   object 
 9   directorate              2010 non-null   object 
 10  grade                    2010 non-null   object 
 11  education                2010 non-null   object 
 12  major                    2010 non-null   object 
 13  pauli                    2010 non-null   int64  
 14  faxtor                  

## 3.3 Load Support Data

In [35]:
# Performance Data
perf = """
SELECT 
    employee_id,
    year as performance_year,
    rating as performance_rating
FROM performance_yearly;
"""

# Strength Data
strength = """
SELECT
    employee_id,
    rank as strength_rank,
    theme as strength_theme
FROM strengths;
"""

# Competency Data
competency = """
SELECT
    c.employee_id,
    c.score as competency_score,
    c.pillar_code as competency_pillar_code,
    comp.pillar_label as competency_pillar_label,
    c.year as competency_year
FROM competencies_yearly c
    LEFT JOIN dim_competency_pillars comp ON c.pillar_code = comp.pillar_code
;
"""

# PAPI Score Data

In [36]:
# Run Queries
df_perf = pd.read_sql(perf, engine)
df_strength = pd.read_sql(strength, engine)
df_competency = pd.read_sql(competency, engine) 

In [ ]:
data_columns = """
    -- Performance Data (from performance_yearly table)
    perf.year as performance_year,
    perf.rating as performance_rating,


    -- Strength (Strengths of each employee)
    str.rank as strength_rank,
    str.theme as strength_theme,

    -- competency_yearly
    comp.score as competency_score,
    comp.pillar_code as competency_pillar_code,
    comp.year as competency_year,

    --PAPI Score
    papi.scale_code as papi_scale_code,
    papi.score as papi_score

    LEFT JOIN performance_yearly perf ON e.employee_id = perf.employee_id
 LEFT JOIN strengths str ON e.employee_id = str.employee_id
    LEFT JOIN competencies_yearly comp ON e.employee_id = comp.employee_id
    LEFT JOIN papi_scores papi ON e.employee_id = papi.employee_id
    
    """

OperationalError: (psycopg2.errors.QueryCanceled) canceling statement due to statement timeout

[SQL: 
SELECT
    -- Employee Data (from employees table)
    e.employee_id, -- Employee ID
    e.fullname, -- Employee name
    e.nip, -- Employee NIP
    e.years_of_service_months, -- Employee years of service in months

    -- Dimension Data (from dim_tables)
    c.name as company_name, -- company name
    a.name as area, -- area name
    p.name as position, -- employee position
    d.name as departement_name, -- departement name
    div.name as division, -- division name
    dir.name as directorate, -- employee directorate
    grd.name as grade,
    edu.name as education,
    mjr.name as major,

    -- Performance Data (from performance_yearly table)
    perf.year as performance_year,
    perf.rating as performance_rating,

    -- Psychological Profile Data (from profiles_psych table)
    prof.pauli,
    prof.faxtor,
    prof.disc,
    prof.disc_word,
    prof.mbti,
    prof.iq,
    prof